<a href="https://www.kaggle.com/code/datascientistsohail/lgbm-regressor-ps-se03ep01?scriptVersionId=115902691" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from lightgbm  import LGBMRegressor


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e1/sample_submission.csv
/kaggle/input/playground-series-s3e1/train.csv
/kaggle/input/playground-series-s3e1/test.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e1/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e1/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e1/sample_submission.csv')
df_train.head()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500


In [3]:
print("Shape of Data: ",df_train.shape)
print("Tuo see if there any null values")
df_train.isnull().sum()

Shape of Data:  (37137, 10)
Tuo see if there any null values


id             0
MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64

In [4]:
y = df_train['MedHouseVal']
df_train.pop('MedHouseVal')
df_train.pop('id')
df_test.pop('id')
X= df_train
X_test = df_test
del df_train
del df_test

In [5]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41


In [6]:
X_test.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,1.7062,35.0,4.966368,1.096539,1318.0,2.844411,39.75,-121.85
1,1.3882,22.0,4.187035,1.098229,2296.0,3.180218,33.95,-118.29
2,7.7197,21.0,7.129436,0.959276,1535.0,2.888889,33.61,-117.81
3,4.6806,49.0,4.769697,1.048485,707.0,1.743590,34.17,-118.34
4,3.1284,25.0,3.765306,1.081633,4716.0,2.003827,34.17,-118.29


In [7]:
print(X.shape)
print(X_test.shape)

(37137, 8)
(24759, 8)


### Cross Validation for the LGBM Regressor
**Assume that you have X and y as your input data and labels**

create a KFold object with 5 splits

In [8]:
folds = KFold(n_splits=5)
model = LGBMRegressor(boosting_type='gbdt', num_leaves=31, learning_rate=0.01, n_estimators=1500)
accuracies = []
y_preds = np.zeros(len(X_test))
# loop through the folds and split the data into train and test sets
for train_index, valid_index in folds.split(X,y):
    
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
  
    # train your model on the training set and evaluate on the test set
    # create an xgboost regression model
    
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse_val = mean_squared_error(y_valid, preds, squared= False)
    print(rmse_val)
    # store the performance for each fold in a list
    accuracies.append(rmse_val)
    y_preds += model.predict(X_test) / folds.n_splits 
# calculate the average performance across all folds
avg_accuracies = sum(accuracies) / len(accuracies)
print('Average Accuracy', avg_accuracies)

0.5609249714366414
0.5654579723284627
0.5675598742037642
0.5595816746171081
0.5792779028145592
Average Accuracy 0.5665604790801071


In [9]:
print(y_preds.shape)
print(submission.shape)

(24759,)
(24759, 2)


In [10]:
submission['MedHouseVal'] = y_preds
submission.to_csv('submission.csv', index = False)